In [1]:
import os

from folioclient import FolioClient

okapi_url = os.getenv("OKAPI_URL")
user = os.getenv("OKAPI_USER")
password = os.getenv("OKAPI_PASSWORD")
tenant = os.getenv("TENANT")

folio_client = FolioClient(
    okapi_url,
    tenant,
    user,
    password)

## Path 1: Use payment transactions to start
**FOLIO UI uses this query:**
```
query="?query=((fiscalYearId=200bfabe-07c7-4deb-b54e-99d64a3435cb and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)) and transactionType==\"Payment\") sortby transactionDate/sort.descending"
```

We don't need to limit by fiscal year.

In [2]:
query="?query=((fiscalYearId=200bfabe-07c7-4deb-b54e-99d64a3435cb and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)) and transactionType==\"Payment\") sortby transactionDate/sort.descending"
ui_response = folio_client.folio_get("/finance/transactions", query=query)
print(ui_response['totalRecords'])

20


**Get all payment transactions for a digital bookplate fund**

transactionType=Payment

fund ID = UUID of fund

In [3]:
query = {'query':'transactionType=Payment and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)'}
total_transactions = folio_client.folio_get("/finance/transactions", query_params=query, key="totalRecords")
print(total_transactions)

21


**Using folio_get_all function to get all transactions**

folio_get_all cannot take a dict for the query. You must pass a string for query.

In [4]:
query = "?query=transactionType==Payment and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)"
transactions = folio_client.folio_get_all("/finance/transactions", key="transactions", query=query, limit=999)

**Get the invoice line ID from the transaction**

In [5]:
invoice_lines = []
for x in transactions:
    # print(x)
    invoice_lines.append(x['sourceInvoiceLineId'])

print(len(invoice_lines))
print(invoice_lines)

20
['6bcc942c-962c-4443-98ee-2d48b9ee5d1e', '0b9ae0fa-792f-4db2-9254-648dfb185bb8', 'd2954121-7364-4ba2-aa07-05a97c957032', '76d19107-b65a-408c-ad3a-3b0c10c0c0ac', '28d90309-af31-484d-977c-ab45666665f7', 'd384cf0f-dfbf-44ef-a88c-3b1776b3a999', '5c6cffcf-1951-47c9-817f-145cbe931dea', 'cb0baa2d-7dd7-4986-8dc7-4909bbc18ce6', '3676f5ad-e22b-487e-b621-c15f441b0186', 'e0cf3343-121b-46fa-9253-c56b856c938b', '2c022a57-8070-4854-90ef-6cc09d55d114', '373439f7-8cf8-4908-86cc-dad6fd850729', '617f4238-baa4-461c-9aca-ec4b6c842994', '6b76dfec-8fb8-4fa7-9fc3-167603d4713c', 'a87a5114-71f0-4bcf-a799-e56c638c73dc', 'e0911498-0a55-49b7-a01f-d209a6ed8f11', '853b5a6b-4bda-4bbf-8fed-e1cb6b4709f0', 'b07a1c90-de9b-4b94-b4b6-4c66d3244f35', '25d23ff1-5c7a-42e1-b7bd-28be22119d8c', '668d0583-ad80-492e-82a6-a7e61c8fe1a1']


**Get the PO line ID from the invoice line**

Not all invoice lines will have a po line. For instance, if the invoice line is for SHIPPING.

In [6]:
transactions_query_po_lines = []
for x in invoice_lines:
    invoice_line = folio_client.folio_get(f"/invoice/invoice-lines/{x}")
    #print(invoice_line)
    po_line_id = invoice_line.get('poLineId')
    if po_line_id is not None:
        transactions_query_po_lines.append(po_line_id)

print(len(transactions_query_po_lines))
print(transactions_query_po_lines)

18
['87319dc7-a5ea-4380-9b99-b02ec15638ed', '24491205-2f0c-4034-bdbb-f715d0410780', '2fb3c6ce-4541-4c19-8e16-d8ed6dd7a9f9', 'e399f1e9-6d10-4116-98d4-e7d192a00e57', 'e8cef55e-2bdc-4c1c-980d-3c66e493798b', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', 'd55342ce-0a33-4aa2-87c6-5ad6e1a12b75', 'be0af62c-665e-4178-ae13-e3250d89bcc6', '5dc4cfb3-a86f-4cfe-86f9-949fbe84aa17', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', '798596da-12a6-4c6d-8d3a-3bb6c54cb2f1', '7a5888fe-689b-4cfe-a27d-c2675a235203', '9047a74c-1057-4ddb-b52d-d01429b0b450', '1d45bbb7-63e1-47e9-9b45-75a06f2c7590', 'c3159ebb-7d49-4eee-933a-7a63636e4750', '08ad0d3a-a906-4bf7-8dc5-28cf3964daae', '39bbf04f-e729-4681-b805-bff3e4b933ac']


Maybe we don't need to start at transactions because we can query invoice lines on paid invoices with fund distributions for each of the digital bookplate funds.

## Path 2: Use invoice lines to start

In [7]:
query = "?query=fundDistributions==\"*3eb86c5f-c77b-4cc9-8f29-7de7ce313411*\" and invoiceLineStatus==\"Paid\""
invoice_lines = folio_client.folio_get_all("/invoice/invoice-lines", key="invoiceLines", query=query, limit=999)

In [8]:
invoices_query_po_lines = []
for x in invoice_lines:
    invoice_line_id = x['id']
    invoice_line = folio_client.folio_get(f"/invoice/invoice-lines/{invoice_line_id}")
    # print(invoice_line)
    po_line_id = invoice_line.get('poLineId')
    if po_line_id is not None:
        invoices_query_po_lines.append(po_line_id)

print(len(invoices_query_po_lines))
print(invoices_query_po_lines)
    

20
['798596da-12a6-4c6d-8d3a-3bb6c54cb2f1', '7a5888fe-689b-4cfe-a27d-c2675a235203', 'e8cef55e-2bdc-4c1c-980d-3c66e493798b', 'be0af62c-665e-4178-ae13-e3250d89bcc6', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', 'd55342ce-0a33-4aa2-87c6-5ad6e1a12b75', '5dc4cfb3-a86f-4cfe-86f9-949fbe84aa17', '2fb3c6ce-4541-4c19-8e16-d8ed6dd7a9f9', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', '9047a74c-1057-4ddb-b52d-d01429b0b450', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', 'e399f1e9-6d10-4116-98d4-e7d192a00e57', 'e8cef55e-2bdc-4c1c-980d-3c66e493798b', '1d45bbb7-63e1-47e9-9b45-75a06f2c7590', '24491205-2f0c-4034-bdbb-f715d0410780', '39bbf04f-e729-4681-b805-bff3e4b933ac', '87319dc7-a5ea-4380-9b99-b02ec15638ed', 'c3159ebb-7d49-4eee-933a-7a63636e4750', '08ad0d3a-a906-4bf7-8dc5-28cf3964daae']


There really should not be a difference in the number of po lines we get from querying the transactions endpoint versus the invoice lines endpoint. If there is, then it is likely the query is not being passed correctly to okapi.

In [9]:
print("PO Lines from paid transactions via invoice lines:", sorted(transactions_query_po_lines))
print("PO Lines from paid invoices' invoice lines:", sorted(invoices_query_po_lines))
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

additional_po_lines_from_transactions = diff(transactions_query_po_lines, invoices_query_po_lines)
print(additional_po_lines_from_transactions)
additional_po_lines_from_invoice_lines = diff(invoices_query_po_lines, transactions_query_po_lines)
print(additional_po_lines_from_invoice_lines)

PO Lines from paid transactions via invoice lines: ['08ad0d3a-a906-4bf7-8dc5-28cf3964daae', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', '1d45bbb7-63e1-47e9-9b45-75a06f2c7590', '24491205-2f0c-4034-bdbb-f715d0410780', '2fb3c6ce-4541-4c19-8e16-d8ed6dd7a9f9', '39bbf04f-e729-4681-b805-bff3e4b933ac', '5dc4cfb3-a86f-4cfe-86f9-949fbe84aa17', '798596da-12a6-4c6d-8d3a-3bb6c54cb2f1', '7a5888fe-689b-4cfe-a27d-c2675a235203', '87319dc7-a5ea-4380-9b99-b02ec15638ed', '9047a74c-1057-4ddb-b52d-d01429b0b450', 'be0af62c-665e-4178-ae13-e3250d89bcc6', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', 'c18b9e40-ad39-40dd-b918-1bd2b999fdaf', 'c3159ebb-7d49-4eee-933a-7a63636e4750', 'd55342ce-0a33-4aa2-87c6-5ad6e1a12b75', 'e399f1e9-6d10-4116-98d4-e7d192a00e57', 'e8cef55e-2bdc-4c1c-980d-3c66e493798b']
PO Lines from paid invoices' invoice lines: ['08ad0d3a-a906-4bf7-8dc5-28cf3964daae', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', '13e05f54-343e-4daf-acf9-3ae6fc66f5e1', '1d45bbb7-63e1-47e9-9b45-75a06f2c7590', '24491205-2f0c-4034-bdb

## Get the instance ID from PO lines
Some of the po lines don't have an instance ID because they are for a package. Example: https://folio.stanford.edu/orders/lines/view/9047a74c-1057-4ddb-b52d-d01429b0b450?limit=50&offset=0&qindex=poLineNumber&query=55487-1

Outstanding question about what to do with these (report out?).

In [10]:
instances = []
for x in invoices_query_po_lines:
    order_line = folio_client.folio_get(f"/orders/order-lines/{x}")
    instance_id = order_line.get('instanceId')
    if instance_id is None:
        print(f"PO Line {order_line['poLineNumber']} not linked to a folio Instance record.")
    else:
        print(instance_id)
        instances.append(instance_id)



242c6000-8485-5fcd-9b5e-adb60788ca59
0a97a746-87ce-58f1-bb61-ffe47b39aff8
772fe534-028f-5579-b2b0-7079f404cd88
a5a1b279-1a00-5af2-9bbb-b5657fa15078
06660d4f-982d-54e8-b34c-532c268868e1
0410d399-b34d-5830-85d5-ce815dbb3c5d
98f6a7fa-21af-5b08-834b-3f7e97457834
0991463f-5be0-544b-a827-ffc628594a50
1cb88f99-5c59-55e8-a834-e45a6a95f778
PO Line 55487-1 not linked to a folio Instance record.
1cb88f99-5c59-55e8-a834-e45a6a95f778
06660d4f-982d-54e8-b34c-532c268868e1
947cd292-a543-5ba7-b5b2-59f9e2523b1e
772fe534-028f-5579-b2b0-7079f404cd88
7a465bf3-ab73-4c30-bd7b-28a7905a5b3e
e361da4f-5fbe-53d8-9fdc-7bb6ab378490
2816efec-41a1-4d79-803a-95769f33be09
857421b8-5704-43b2-99fd-3a5f285b3f9e
7327db79-7f3c-42e1-9614-afe9ed1ca2a3
8fcdaf3b-808e-4fde-8fa4-d3ad58363c7f


**Get the MARC for each instance to do a PUT to add 979**

Outstanding question: will the 979 be added to the marc-bib mapping for updating the instance json?

We probably could refactor the [OCLCAPIWrapper class](https://github.com/sul-dlss/libsys-airflow/blob/d6b6f1c2b7b0db4b231520edd92ef814bccd3932/libsys_airflow/plugins/data_exports/oclc_api.py#L75) to do a GET to change-manager/parsedRecords since it will give us the data we need to do a PUT (SRS record ID, instance hrid). There is a relatedRecordVersion field from change-manager/parsedRecords that should be the instance version (for optimistic locking) but I think it is a new field and we do not have it populated in our data at that endpoint. It is a required field so we need to get it from inventory/instances.

In [11]:
for instance_id in instances:
    instance_version = folio_client.folio_get(f"/inventory/instances/{instance_id}").get('_version')
    query = {'externalId': f"{instance_id}"}
    parsed_record = folio_client.folio_get("/change-manager/parsedRecords", query_params=query)
    record_fields = parsed_record['parsedRecord']['content']['fields']
    record_979s = []
    field_979 = next((field for field in record_fields if field.get('979')), None)
    record_979s.append(field_979)
    print(f"id: {parsed_record['id']}", f"instance id: {parsed_record['externalIdsHolder']['instanceId']}", f"instance hrid: {parsed_record['externalIdsHolder']['instanceHrid']}", f"record state: {parsed_record['recordState']}", f"version: {parsed_record.get('relatedRecordVersion', instance_version)}", f"979 fields: {record_979s}", sep="\n")

id: 589e1cd5-bf64-5724-bce7-82dc13a033fe
instance id: 242c6000-8485-5fcd-9b5e-adb60788ca59
instance hrid: a470860
record state: ACTUAL
version: 4
979 fields: [{'979': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'f': 'ABBOTT'}, {'b': 'druid:ws066yy0421'}, {'c': 'ws066yy0421_00_0001.jp2'}, {'d': 'The The Donald P. Abbott Fund for Marine Invertebrates'}]}}]
id: e51ece5e-d620-5227-babf-c32024751f6b
instance id: 0a97a746-87ce-58f1-bb61-ffe47b39aff8
instance hrid: a358123
record state: ACTUAL
version: 3
979 fields: [{'979': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'f': 'ABBOTT'}, {'b': 'druid:ws066yy0421'}, {'c': 'ws066yy0421_00_0001.jp2'}, {'d': 'The The Donald P. Abbott Fund for Marine Invertebrates'}]}}]
id: 7ab8a798-5a53-5154-8d3a-ffe573b640a5
instance id: 772fe534-028f-5579-b2b0-7079f404cd88
instance hrid: a405027
record state: ACTUAL
version: 3
979 fields: [{'979': {'ind1': ' ', 'ind2': ' ', 'subfields': [{'f': 'ABBOTT'}, {'b': 'druid:ws066yy0421'}, {'c': 'ws066yy0421_00_0001.jp2'}, {'d